## Training GPT-2 Model on Path Sequences to Perform Navigation Tasks
This script trains/fine-tunes a GPT-2 model on path sequences in artificial grid environments with randomly generated nouns representing locations on a grid.

This script has the following parts:

* Generation of path sequences in artificial environments
Optimal paths and suboptimal paths are separately generated in the training data. For optimal paths, the shortest paths from any starting and ending locations are generated, therefore the training data includes all the possible optimal paths. For suboptimal paths, only a subset is randomly selected, because the number of possible suboptimal paths greatly outnumber optimal paths as the grid size increases. 

* Train/fine-tune GPT-2 model on the generated path sequences




#### Installation / imports:

In [1]:
# # Connect to Google drive
# from google.colab import drive
# drive.mount('/content/drive')
# %cd "/content/drive/My Drive/modelling_spatial_navigation/gpt2small_dual_mode_v12_6000_4by4grids"

In [2]:
# python and pip version check
! which python
! which pip
! python --version

/cs/student/projects1/aibh/2024/cbaumgar/.venv/bin/python
/cs/student/projects1/aibh/2024/cbaumgar/.venv/bin/pip
Python 3.9.21


In [3]:
! pip install git+https://github.com/huggingface/transformers --upgrade
! pip install accelerate evaluate wonderwords simpletransformers --upgrade
! pip install huggingface_hub --upgrade

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-w6wu31ub
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-w6wu31ub
  Resolved https://github.com/huggingface/transformers to commit d7b87b415a5dd4a3152051e1a0abd098a02c5bfa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 53.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731

### Generate training and test data.

In [3]:
import random
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random
import string
import os
import re
import glob
import torch
from wonderwords import RandomWord
import os
import gc
import pickle
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from itertools import permutations
import logging
from random import shuffle
from matplotlib import pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import math

os.environ["WANDB_DISABLED"] = "true"

Random_Word = RandomWord()

# Function to generate a random grid of nouns
def create_unique_random_grid(nouns, size):
    random_nouns = random.sample(nouns, size * size)
    return [random_nouns[i * size:(i + 1) * size] for i in range(size)]

# Function to generate all start-end combinations for a given grid
def generate_start_end_permutations(size):
    combinations = []
    for start_x in range(size):
        for start_y in range(size):
            for end_x in range(size):
                for end_y in range(size):
                    if (start_x, start_y) != (end_x, end_y):  # Exclude combinations where start == end
                        combinations.append(((start_x, start_y), (end_x, end_y)))
    return combinations

# Function to generate all possible optimal paths for given start-end combinations
def generate_path_permutations(combinations):
    paths = []
    for start, end in combinations:
        horizontal_steps = abs(end[1] - start[1])  # Difference in columns
        vertical_steps = abs(end[0] - start[0])    # Difference in rows
        steps = ['H'] * horizontal_steps + ['V'] * vertical_steps

        unique_permutations = set(permutations(steps))

        for perm in unique_permutations:
            path = [start]
            current_pos = list(start)
            for step in perm:
                if step == 'H':
                    current_pos[1] += 1 if end[1] > start[1] else -1
                else:  # 'V'
                    current_pos[0] += 1 if end[0] > start[0] else -1
                path.append(tuple(current_pos))
            # Append each path to the paths list
            paths.append({'start': start, 'end': end, 'path': path})

    return paths

# Function to generate nouns
def generate_nouns(size):
    Random_Word = RandomWord()
    nouns = set()
    while len(nouns) < size * size:
        # Fetching a random noun and replacing spaces with underscores
        word = Random_Word.word(include_parts_of_speech=["nouns"]).replace(" ", "_")
        if word:  # Ensure that 'None' isn't added to the set if no word is returned
            nouns.add(word)
    return list(nouns)

def shuffle_stimuli(stimuli):
    random.shuffle(stimuli)
    return stimuli

def get_direction(prev_coord, coord):
    if coord[0] == prev_coord[0]:
        if coord[1] > prev_coord[1]:
            return "R"
        else:
            return "L"
    else:
        if coord[0] > prev_coord[0]:
            return "D"
        else:
            return "U"

def generate_all_suboptimal_paths(size):

    def is_valid_move(start, end):
        """Check if the move from start to end is valid (adjacent cells)."""
        return abs(start[0] - end[0]) + abs(start[1] - end[1]) == 1

    def is_suboptimal(path, start, end):
        """Check if the path is suboptimal (longer than the Manhattan distance)."""
        optimal_length = abs(end[0] - start[0]) + abs(end[1] - start[1])
        return len(path) > optimal_length + 1  # Must be longer than the optimal path

    def generate_all_possible_paths(size):
        """Generate all possible valid paths for a given grid size."""
        coords = [(x, y) for x in range(size) for y in range(size)]
        all_paths = []
        for start in coords:
            for end in coords:
                if start != end:
                    queue = [[start]]
                    while queue:
                        path = queue.pop(0)
                        current_pos = path[-1]
                        if current_pos == end:
                            if is_suboptimal(path, start, end):
                                all_paths.append(path)
                        else:
                            for move in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # Right, Down, Left, Up
                                next_pos = (current_pos[0] + move[0], current_pos[1] + move[1])
                                if (0 <= next_pos[0] < size and 0 <= next_pos[1] < size and
                                    next_pos not in path and is_valid_move(current_pos, next_pos)):
                                    queue.append(path + [next_pos])
        return all_paths

    """Generate all suboptimal paths for a given grid size."""
    suboptimal_paths = []
    all_possible_paths = generate_all_possible_paths(size)
    for path in all_possible_paths:
        suboptimal_paths.append({'start': path[0], 'end': path[-1], 'path': path})
    return suboptimal_paths

def get_movement_options(coord, grid_size, prev_coord):
    options = ["U", "D", "L", "R"]
    moves = {
        "U": (coord[0] - 1, coord[1]),
        "D": (coord[0] + 1, coord[1]),
        "L": (coord[0], coord[1] - 1),
        "R": (coord[0], coord[1] + 1)
    }
    valid_moves = []
    for option in options:
        move = moves[option]
        if 0 <= move[0] < grid_size and 0 <= move[1] < grid_size and move != prev_coord:
            valid_moves.append(option)
        else:
            valid_moves.append("NA")
    return f"[{' '.join(valid_moves)}]"

def generate_shortest_path_string(grid, path_coords, probability):
    if not path_coords:
        return ""

    size = len(grid)
    start, end = path_coords[0], path_coords[-1]
    path_string = f"MODE: Shortest, START: {grid[start[0]][start[1]]}, END: {grid[end[0]][end[1]]}, PATH: "
    prev_coord = None
    first_step = True

    for i, coord in enumerate(path_coords):
        if coord == start:
            path_string += grid[start[0]][start[1]]
        else:
            direction = get_direction(prev_coord, coord)
            if first_step:
                movement_options = get_movement_options(prev_coord, size, path_coords[i - 2] if i > 1 else None)
                path_string += f" {movement_options} {direction}"
                first_step = False
            else:
                path_string += f" {direction}"

            if coord == end:
                path_string += f" {grid[coord[0]][coord[1]]}"
            else:
                if random.random() < probability:
                    path_string += f" {grid[coord[0]][coord[1]]}"
                else:
                    path_string += " FORGOT"

        prev_coord = coord

    return path_string

def generate_foraging_path_string(grid, path_coords, probability):
    if not path_coords:
        return ""

    size = len(grid)
    start, end = path_coords[0], path_coords[-1]
    path_string = f"MODE: Foraging, START: {grid[start[0]][start[1]]}, END: {grid[end[0]][end[1]]}, PATH: "
    prev_coord = None
    first_step = True

    for i, coord in enumerate(path_coords):
        if coord == start:
            path_string += grid[start[0]][start[1]]
        else:
            direction = get_direction(prev_coord, coord)
            if first_step:
                movement_options = get_movement_options(prev_coord, size, path_coords[i - 2] if i > 1 else None)
                path_string += f" {movement_options} {direction}"
                first_step = False
            else:
                path_string += f" {direction}"

            if coord == end:
                path_string += f" {grid[coord[0]][coord[1]]}"
            else:
                if random.random() < probability:
                    path_string += f" {grid[coord[0]][coord[1]]}"
                else:
                    path_string += " FORGOT"

        prev_coord = coord

    return path_string

size = 4  # Grid size
shortest_paths_iterations = 1 # How many times each optimal path is included in the training data
num_suboptimal = 744 # How many suboptimal paths to be randomly selected for a single grid environment
probability = 1 # The probability of forgetting intermediate locations in a given path

training_strs = []
for i in range(3000): # Number of grid environments in training data
    nouns_list = generate_nouns(size)
    grid = create_unique_random_grid(nouns_list, size)
    start_end_combinations = generate_start_end_permutations(size)

    shortest_paths = generate_path_permutations(start_end_combinations)
    suboptimal_paths = generate_all_suboptimal_paths(size)

    generated_path_strings = []

    # Add all shortest paths for a specified number of times (shortest_paths_iterations) to the list of generated path strings
    for _ in range(shortest_paths_iterations):
        for path in shortest_paths:
            generated_path_strings.append(generate_shortest_path_string(grid, path['path'], probability))

    # From all possible subtoptimal paths, randomly select a specified number and add to the list
    selected_paths = random.sample(suboptimal_paths, num_suboptimal)
    for path in selected_paths:
        generated_path_strings.append(generate_foraging_path_string(grid, path['path'], probability))

    # Shuffle the list of generated path strings
    generated_path_strings = shuffle_stimuli(generated_path_strings)

    # Append the generated path strings to the training strings list
    training_strs += generated_path_strings

    # Print the iteration number
    if i % 100 == 0:
        print(f"Iteration {i} completed.")

testing_strs = []
for i in range(30): # Number of grid environments in testing data
    nouns_list = generate_nouns(size)
    grid = create_unique_random_grid(nouns_list, size)
    start_end_combinations = generate_start_end_permutations(size)

    shortest_paths = generate_path_permutations(start_end_combinations)
    suboptimal_paths = generate_all_suboptimal_paths(size)

    generated_path_strings = []

    # Add all shortest paths for a specified number of times (shortest_paths_iterations) to the list of generated path strings
    for _ in range(shortest_paths_iterations):
        for path in shortest_paths:
            generated_path_strings.append(generate_shortest_path_string(grid, path['path'], probability))

    # From all possible subtoptimal paths, randomly select a specified number and add to the list
    selected_paths = random.sample(suboptimal_paths, num_suboptimal)
    for path in selected_paths:
        generated_path_strings.append(generate_foraging_path_string(grid, path['path'], probability))

    # Shuffle the list of generated path strings
    generated_path_strings = shuffle_stimuli(generated_path_strings)

    # Append the generated path strings to the testing strings list
    testing_strs += generated_path_strings

# Print the total number of training and testing path strings
print(f"\n{len(training_strs)} paths generated for pre-training.")
print(f"{len(testing_strs)} paths generated for testing.")

/cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
/cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Iteration 0 completed.
Iteration 100 completed.
Iteration 200 completed.
Iteration 300 completed.
Iteration 400 completed.
Iteration 500 completed.
Iteration 600 completed.
Iteration 700 completed.
Iteration 800 completed.
Iteration 900 completed.
Iteration 1000 completed.
Iteration 1100 completed.
Iteration 1200 completed.
Iteration 1300 completed.
Iteration 1400 completed.
Iteration 1500 completed.
Iteration 1600 completed.
Iteration 1700 completed.
Iteration 1800 completed.
Iteration 1900 completed.
Iteration 2000 completed.
Iteration 2100 completed.
Iteration 2200 completed.
Iteration 2300 completed.
Iteration 2400 completed.
Iteration 2500 completed.
Iteration 2600 completed.
Iteration 2700 completed.
Iteration 2800 completed.
Iteration 2900 completed.

4464000 paths generated for pre-training.
44640 paths generated for testing.


Write training and testing data into .txt files, and count number of tokens:

In [4]:
!rm -rf spatial_model
!mkdir spatial_model

text_file = open("spatial_model/train.txt", "w")
n = text_file.write('\n'.join(training_strs))
text_file.close()

text_file = open("spatial_model/test.txt", "w")
n = text_file.write('\n'.join(testing_strs))
text_file.close()


rm: cannot remove 'spatial_model': Directory not empty
mkdir: cannot create directory ‘spatial_model’: File exists


### Train/Fine-Tune GPT-2 Model

The function below runs a script to fine-tune a gpt-2 model on the arbitrary stimuli.

The name_or_path argument is which model to fine-tune from. In the pre-training stage, this will be set to 'gpt2'.

In [5]:
import random
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random
import string
import os
import re
import glob
import torch
from wonderwords import RandomWord
import os
import gc
import pickle
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from itertools import permutations
import logging
from random import shuffle
from matplotlib import pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import math

os.environ["WANDB_DISABLED"] = "true"

# Check CUDA, MPS, and CPU availability
if torch.cuda.is_available():
    print("CUDA available. Using CUDA.")
    device = torch.device("cuda")

elif torch.backends.mps.is_available():
    print("MPS available. Using MPS.")
    device = torch.device("mps")

else:
    print("CUDA and MPS not available. Using CPU.")
    device = torch.device("cpu")

CUDA available. Using CUDA.


In [6]:
torch.cuda.empty_cache()
gc.collect()

from transformers import logging
logging.set_verbosity_info()


In [7]:
# def train_model_script(name_or_path='spatial_model',
#                        num_epochs=1,
#                        output_dir='./models',
#                        save_strategy="steps",
#                        save_steps=111600,  # ≈ every 10% of training
#                        lr=5e-05,
#                        train_batch_size=32,
#                        eval_batch_size=32,
#                        block_size=1024):
#     torch.cuda.empty_cache()
#     gc.collect()
#     ! python ./run_clm.py \
#         --model_name_or_path {name_or_path} \
#         --train_file {os.path.join(output_dir, 'train.txt')} \
#         --validation_file {os.path.join(output_dir, 'test.txt')} \
#         --per_device_train_batch_size {train_batch_size} \
#         --per_device_eval_batch_size {eval_batch_size} \
#         --do_train \
#         --do_eval \
#         --output_dir {output_dir} \
#         --overwrite_output_dir \
#         --num_train_epochs {num_epochs} \
#         --save_strategy steps \
#         --save_steps {save_steps} \
#         --learning_rate {lr} \
#         --block_size {block_size}
        


In [ ]:
def train_model_script(name_or_path='spatial_model',
                       num_epochs=1,
                       output_dir='./spatial_model',
                       save_strategy="steps",
                       save_steps=111600,
                       lr=5e-5,
                       train_batch_size=32,
                       eval_batch_size=32,
                       block_size=1024):
    import torch, gc, os
    torch.cuda.empty_cache()
    gc.collect()

    # Define model and tokenizer names explicitly
    model_type = "gpt2"
    tokenizer_name = "gpt2"

    # Assemble the training command
    os.system(f"""
    python ./run_clm.py \
        --model_type {model_type} \
        --tokenizer_name {tokenizer_name} \
        --train_file {os.path.join(output_dir, 'train.txt')} \
        --validation_file {os.path.join(output_dir, 'test.txt')} \
        --per_device_train_batch_size {train_batch_size} \
        --per_device_eval_batch_size {eval_batch_size} \
        --do_train \
        --do_eval \
        --evaluation_strategy "steps" \
        --eval_steps {save_steps} \
        --output_dir {output_dir} \
        --overwrite_output_dir \
        --num_train_epochs {num_epochs} \
        --save_strategy {save_strategy} \
        --save_steps {save_steps} \
        --learning_rate {lr} \
        --block_size {block_size}
    """)


In [9]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

with open("spatial_model/train.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

print(f"Total lines: {len(lines)}")

sample = lines[:100]
token_counts = [len(tokenizer.encode(line.strip())) for line in sample]
avg_tokens_per_line = sum(token_counts) / len(token_counts)

print(f"Average tokens per line (estimated): {avg_tokens_per_line:.2f}")

total_estimated_tokens = len(lines) * avg_tokens_per_line
print(f"Estimated total tokens: {int(total_estimated_tokens):,}")
block_size = 1024
train_batch_size = 4

num_blocks = int(total_estimated_tokens) // block_size
num_steps = num_blocks // train_batch_size

print(f"Approx. training steps per epoch: {num_steps:,}")


loading file vocab.json from cache at /cs/student/msc/aibh/2024/cbaumgar/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/vocab.json
loading file merges.txt from cache at /cs/student/msc/aibh/2024/cbaumgar/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /cs/student/msc/aibh/2024/cbaumgar/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer_config.json
loading file tokenizer.json from cache at /cs/student/msc/aibh/2024/cbaumgar/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/tokenizer.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /cs/student/msc/aibh/2024/cbaumgar/.cache/huggingface/hub/models--gpt2/snap

Total lines: 4464000
Average tokens per line (estimated): 37.67
Estimated total tokens: 168,158,880
Approx. training steps per epoch: 41,054


In [17]:
# Train GPT-2 model for one epoch
train_model_script(name_or_path='gpt2',
                   num_epochs=1,
                   output_dir='./spatial_model',
                   save_steps=4471,
                   lr=5e-05,
                   train_batch_size=4,
                   eval_batch_size=4,
                   block_size=1024)

/cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


06/09/2025 15:56:09 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
06/09/2025 15:56:09 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
d

Using custom data configuration default-f886da7084900083
Loading Dataset Infos from /cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/datasets/packaged_modules/text
Generating dataset text (/cs/student/projects1/aibh/2024/cbaumgar/hf_cache/datasets/text/default-f886da7084900083/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99)
Checksum Computation took 0.0 min
Generating train split
Generating train split: 0 examples [00:00, ? examples/s]

06/09/2025 15:56:09 - INFO - datasets.builder - Using custom data configuration default-f886da7084900083
06/09/2025 15:56:09 - INFO - datasets.info - Loading Dataset Infos from /cs/student/projects1/aibh/2024/cbaumgar/.venv/lib64/python3.9/site-packages/datasets/packaged_modules/text
06/09/2025 15:56:09 - INFO - datasets.builder - Generating dataset text (/cs/student/projects1/aibh/2024/cbaumgar/hf_cache/datasets/text/default-f886da7084900083/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99)
06/09/2025 15:56:09 - INFO - datasets.builder - Downloading and preparing dataset text/default to /cs/student/projects1/aibh/2024/cbaumgar/hf_cache/datasets/text/default-f886da7084900083/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99...
06/09/2025 15:56:09 - INFO - datasets.download.download_manager - Downloading took 0.0 min
06/09/2025 15:56:09 - INFO - datasets.download.download_manager - Checksum Computation took 0.0 min
06/09/2025 15:56:09 - INFO - 

Generating train split: 4464000 examples [00:05, 772158.83 examples/s]
Generating validation split
Generating validation split: 44640 examples [00:00, 506874.14 examples/s]
Unable to verify splits sizes.
Dataset text downloaded and prepared to /cs/student/projects1/aibh/2024/cbaumgar/hf_cache/datasets/text/default-f886da7084900083/0.0.0/37eaf37ac90527a7fd768c94b312ee84f8815c9b7ac00acf81c1c364e8392f99. Subsequent calls will reuse this data.


...

{'train_runtime': 11237.5509, 'train_samples_per_second': 15.896, 'train_steps_per_second': 3.974, 'train_loss': 0.7102367408212769, 'epoch': 1.0}


[INFO|modeling_utils.py:3840] 2025-06-09 19:11:00,726 >> Model weights saved in ./spatial_model/model.safetensors
[INFO|tokenization_utils_base.py:2525] 2025-06-09 19:11:00,728 >> tokenizer config file saved in ./spatial_model/tokenizer_config.json
[INFO|tokenization_utils_base.py:2534] 2025-06-09 19:11:00,757 >> Special tokens file saved in ./spatial_model/special_tokens_map.json
[INFO|trainer.py:4327] 2025-06-09 19:11:00,902 >> 
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-06-09 19:11:00,903 >>   Num examples = 1767
[INFO|trainer.py:4332] 2025-06-09 19:11:00,903 >>   Batch size = 4


***** train metrics *****
  epoch                    =        1.0
  total_flos               = 86940156GF
  train_loss               =     0.7102
  train_runtime            = 3:07:17.55
  train_samples            =     178634
  train_samples_per_second =     15.896
  train_steps_per_second   =      3.974
06/09/2025 19:11:00 - INFO - __main__ - *** Evaluate ***


100%|██████████| 442/442 [00:41<00:00, 10.75it/s]
[INFO|modelcard.py:450] 2025-06-09 19:11:42,487 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8304148887970565}]}


***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.8304
  eval_loss               =     0.8292
  eval_runtime            = 0:00:41.57
  eval_samples            =       1767
  eval_samples_per_second =     42.506
  eval_steps_per_second   =     10.632
  perplexity              =     2.2915


In [11]:
import random
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random
import string
import os
import re
import glob
import torch
from wonderwords import RandomWord
import os
import gc
import pickle
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from itertools import permutations
import logging
from random import shuffle
from matplotlib import pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import math

os.environ["WANDB_DISABLED"] = "true"

# Check CUDA, MPS, and CPU availability
if torch.cuda.is_available():
    print("CUDA available. Using CUDA.")
    device = torch.device("cuda")

elif torch.backends.mps.is_available():
    print("MPS available. Using MPS.")
    device = torch.device("mps")

else:
    print("CUDA and MPS not available. Using CPU.")
    device = torch.device("cpu")

CUDA available. Using CUDA.


In [3]:
import random
import pandas as pd
import networkx as nx
import logging
from random import shuffle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import random
import string
import os
import re
import glob
import torch
from wonderwords import RandomWord
import os
import gc
import pickle
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from itertools import permutations
import logging
from random import shuffle
from matplotlib import pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import math

os.environ["WANDB_DISABLED"] = "true"

# Check CUDA, MPS, and CPU availability
if torch.cuda.is_available():
    print("CUDA available. Using CUDA.")
    device = torch.device("cuda")

elif torch.backends.mps.is_available():
    print("MPS available. Using MPS.")
    device = torch.device("mps")

else:
    print("CUDA and MPS not available. Using CPU.")
    device = torch.device("cpu")

CUDA available. Using CUDA.


If resume training is needed

In [12]:
# def resume_training_script(name_or_path='./spatial_model',
#                            num_epochs=4,  # Number of additional epochs
#                            output_dir='./clm_script',
#                            lr=5e-05,
#                            train_batch_size=1,
#                            eval_batch_size=1,
#                            block_size=1024):
#     torch.cuda.empty_cache()
#     gc.collect()
#     ! python ./run_clm.py \
#         --model_name_or_path {name_or_path} \
#         --train_file {os.path.join(output_dir, 'train.txt')} \
#         --validation_file {os.path.join(output_dir, 'test.txt')} \
#         --per_device_train_batch_size {train_batch_size} \
#         --per_device_eval_batch_size {eval_batch_size} \
#         --do_train \
#         --do_eval \
#         --output_dir {output_dir} \
#         --overwrite_output_dir \
#         --num_train_epochs {num_epochs} \
#         --save_strategy 'epoch' \
#         --learning_rate {lr} \
#         --block_size {block_size} \
#         --resume_from_checkpoint {name_or_path}


In [13]:
# # Resume training GPT-2 Large model for more epochs
# resume_training_script(name_or_path='./spatial_model',
#                        num_epochs=1,  # Train for 2 additional epochs
#                        output_dir='./spatial_model_2',
#                        lr=5e-05,
#                        train_batch_size=1,
#                        eval_batch_size=1,
#                        block_size=1024)


The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


#### Save Training Log File

In [19]:
# Path to the directory in your Google Drive where you want to save the file
file_path = 'training_output.log'

# Open the file in write mode and write the copied output
with open(file_path, 'w') as file:
    file.write("""



""")


In [ ]:
import os

# home folder out of space, set environment variables to use a different cache directory
os.environ["TRANSFORMERS_CACHE"] = "/cs/student/projects1/aibh/2024/cbaumgar/hf_cache"
os.environ["HF_HOME"] = "/cs/student/projects1/aibh/2024/cbaumgar/hf_cache"


In [ ]:
import os
os.environ["TRANSFORMERS_CACHE"] = "/cs/student/projects1/aibh/2024/cbaumgar/hf_cache"


